In [1]:
# imports
import pandas as pd
import os
import kagglehub
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import scipy

# import kaggle 

/Users/kai/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/kai/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pip install scipy

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# imports
import pandas as pd
# import os
# import kagglehub
import numpy as np

import networkx as nx
import matplotlib.pyplot as plt
import scipy

# import kaggle 

In [ ]:
import pickle

path1 = "../dataframes/edges_df_sorted.pk1"
with open(path1, "rb") as f:
    edges_df_sorted = pickle.load(f)

path2= "../dataframes/edges_df.pk1"
with open(path2, "rb") as f:
    edges_df = pickle.load(f)

path3= "../dataframes/df_aggregated.pk1"
with open(path3, "rb") as f:
    df_aggregated = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'edges_df.pk1'

In [ ]:
edges_df

In [ ]:
#ignore, didn't know 'pain after an operation' would be a disease
disease_of_interest = "pain after an operation"

filtered_edges = edges_df[(edges_df["Disease1"] == disease_of_interest) | (edges_df["Disease2"] == disease_of_interest)]

print(filtered_edges)

In [ ]:
G = nx.Graph()

for _, row in edges_df.iterrows():
    G.add_edge(row["Disease1"], row["Disease2"], weight=row["Weight"])

print("Nodes:", G.number_of_nodes())

print("Edges:", G.number_of_edges())
print("Network Density" , nx.density(G))
#Network density is 0.22, which is fairly low. 1.0 would be a fully connected graph
#There are some distinct cluters, some diseases share symptoms, and others are only connected to a few
#Some diseases have a specific symptom profile



## Degree Distribution

In [ ]:
# Compute degree distribution
degrees = [G.degree(n) for n in G.nodes()]
average_degree = np.mean(degrees)

In [ ]:
# Plot degree distribution
plt.hist(degrees, bins=20)
plt.xlabel("Degree")
plt.ylabel("Frequency")
plt.title("Degree Distribution of Diseases")
plt.show()

In [ ]:
# Plot histogram (linear scale)
plt.figure(figsize=(6, 4))
plt.hist(degrees, bins=20, color='b', alpha=0.6, edgecolor='black')
plt.xlabel("Degree")
plt.ylabel("Frequency")
plt.title("Degree Distribution of Diseases")
plt.figtext(0.15, 0.85, f'Avg Degree: {average_degree:.2f}', fontsize=12)
plt.show()

In [ ]:
# Log-Log Plot (to check power-law behavior)
plt.figure(figsize=(6, 4))
plt.hist(degrees, bins=np.logspace(np.log10(min(degrees)), np.log10(max(degrees)), 20), color='g', alpha=0.6, edgecolor='black')
plt.xscale("log")
plt.yscale("log")
plt.xlabel("Degree (log)")
plt.ylabel("Frequency (log)")
plt.title("Log-Log Degree Distribution")
plt.show()

In [ ]:
# Fit a power-law distribution
fit = powerlaw.Fit(degrees)

In [ ]:
# Plot with fitted power-law
fig = plt.figure(figsize=(6, 4))
fit.plot_pdf(color='b', linewidth=2, label="Empirical Data")
fit.power_law.plot_pdf(color='r', linestyle="--", label="Power-Law Fit")
plt.xlabel("Degree")
plt.ylabel("Probability Density")
plt.legend()
plt.title("Power-Law Fit to Degree Distribution")
plt.show()


In [ ]:
# Print estimated power-law exponent
print(f"Estimated power-law exponent: {fit.power_law.alpha:.2f}")
print(f"Power-law fit xmin: {fit.power_law.xmin}")

# Basic Analysis

In [ ]:
# extracted from networkX cpsc 572 lecture winter 2025
N = len(G)
L = G.size()
degrees = [G.degree(node) for node in G]
kmin = min(degrees)
kmax = max(degrees)

In [ ]:
print("Number of nodes: ", N)
print("Number of edges: ", L)
print()
print("Average degree: ", 2*L/N)
print("Average degree (alternate calculation)", np.mean(degrees))
print()
print("Minimum degree: ", kmin)
print("Maximum degree: ", kmax)

## Plotting Degree Distribution

### Log-Log Plot

In [ ]:
# Get 10 logarithmically spaced bins between kmin and kmax
# buckets un from minimum degree to maximum degree
bin_edges = np.logspace(np.log10(kmin), np.log10(kmax), num=10)

# histogram the data into these bins
density, _ = np.histogram(degrees, bins=bin_edges, density=True)

In [ ]:
import matplotlib.ticker as ticker

In [ ]:
fig = plt.figure(figsize=(6,4))

# "x" should be midpoint (IN LOG SPACE) of each bin
log_be = np.log10(bin_edges)
x = 10**((log_be[1:] + log_be[:-1])/2)

plt.loglog(x, density, marker='o', linestyle='none')
plt.xlabel(r"degree $k$", fontsize=16)
plt.ylabel(r"$P(k)$", fontsize=16)

# Adding a line for better visual 
plt.loglog(x, density, linestyle='-', color='gray', linewidth=0.5, alpha=0.6, label="Trend Line")

# remove right and top boundaries because they're ugly
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')

# Adjusting x-axis numbers for readability
custom_ticks = [50, 100, 150, 200]  
ax.set_xticks(custom_ticks)
ax.set_xticklabels(custom_ticks)  # Ensure they display as integers

ax.xaxis.set_major_formatter(ticker.ScalarFormatter())
ax.tick_params(axis='x', which='major', labelsize=12, rotation=30)  


# Show the plot
plt.show()

### Linear-Linear Plot

In [ ]:
# CODE BELOW TAKEN FROM CPSC 572 LECTURE ON NETWORKX (WINTER 2025)
# Get 20 logarithmically spaced bins between kmin and kmax
bin_edges = np.linspace(kmin, kmax, num=10)  # number of bins (num=10) can be changed

# histogram the data into these bins
density, _ = np.histogram(degrees, bins=bin_edges, density=True)

In [ ]:
fig = plt.figure(figsize=(6,4))

# "x" should be midpoint (IN LOG SPACE) of each bin
log_be = np.log10(bin_edges)
x = 10**((log_be[1:] + log_be[:-1])/2)

plt.plot(x, density, marker='o', linestyle='none')
plt.xlabel(r"degree $k$", fontsize=16)
plt.ylabel(r"$P(k)$", fontsize=16)

# Adding a line for better visual 
plt.plot(x, density, linestyle='-', color='gray', linewidth=0.5, alpha=0.6, label="Trend Line")

# remove right and top boundaries
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')

# Show the plot
plt.show()

# Visualizing the Network

In [ ]:
import random

### Using force-based or "spring" layout algorithm for 50% of dataset

In [ ]:
# Subsample about 25% of the nodes
total_nodes = len(G.nodes())
sampled_nodes = int(total_nodes * 0.25) # Extract 25% of all nodes
final_sampled_nodes = random.sample(G.nodes(), sampled_nodes)

In [ ]:
# Create a subgraph of nodes
subgraph = G.subgraph(final_sampled_nodes)

# Visualize the subgraph using Force-Based Layout
plt.figure(figsize=(8, 8))
plt.title("Force-Based Layout Using 25% Subsample")
pos_force = nx.spring_layout(subgraph, seed=42)  # force-directed layout
nx.draw(subgraph, pos=pos_force, with_labels=False, node_size=50, edge_color="black")
plt.show()

### Using force-based or "spring" layout algorithm for 10% of dataset

In [ ]:
# Subsample about 10% of the nodes
total_nodes = len(G.nodes())
sampled_nodes = int(total_nodes * 0.10) # Extract 10% of all nodes
final_sampled_nodes = random.sample(G.nodes(), sampled_nodes)

In [ ]:
# Create a subgraph of nodes
subgraph = G.subgraph(final_sampled_nodes)

# Visualize the subgraph using Force-Based Layout
plt.figure(figsize=(8, 8))
plt.title("Force-Based Layout Using 10% Subsample")
pos_force = nx.spring_layout(subgraph, seed=42)  # force-directed layout
nx.draw(subgraph, pos=pos_force, with_labels=False, node_size=50, edge_color="black")
plt.show()

### Using the fcircular layout algorithm using 10% of nodes

In [ ]:
plt.figure(figsize=(10, 10))
plt.title("Circular Layout Using 10% Subsample")
pos_circular = nx.circular_layout(subgraph)  # circular layout
nx.draw(subgraph, pos=pos_circular, with_labels=False, node_size=50, edge_color="black")
plt.show()

In [ ]:
pip install python-louvain


In [ ]:
import community as community_louvain

# Detect communities using Louvain method
partition = community_louvain.best_partition(G)

# Color nodes based on community detection results
node_color = [partition[node] for node in subgraph.nodes()]

# Plot with community-based colors
pos = nx.spring_layout(G)
nx.draw(subgraph, pos, node_color=node_color, cmap=plt.cm.RdYlBu, node_size=300)
plt.title("Community-Louvain Visual Using 10% Subsample")
plt.show()


In [ ]:
degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000)

In [ ]:
print("Top 5 diseases by Degree Centrality:", sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:5])
#these diseases have the highest number of direct symptom based connections to other diseases

#Drug reaction:
    #high connectivity suggests drug-induced conditions are frequently misdiagnosed as other illnesses.
    
#Pain after an operation:
    #Post operation pain is common across many surgical procedures and may be mistaken for underlying infections or some other issues
    
#Shingles:
    #High centrality suggests it may often be confused with other viral infections or autoimmune conditions.
    
#Acute Stress Reaction:
    #indicates stress-related symptoms might contribute to misdiagnosis of physical diseases.
    



In [ ]:
print("Top 5 diseases by Betweenness Centrality:", sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:5])

#Shingles:
    #Shingles has symptoms (nerve pain, rashes, fever) that overlap with both viral infections and autoimmune conditions.
    #This suggests shingles acts as a key diagnostic confusion point, potentially being mistaken for other neurological, dermatological, or viral illnesses.
    
#Parasitic disease:
    #Many parasitic diseases have vague symptoms (abdominal pain, diarrhea, fatigue), which means they might connect conditions like food poisoning, viral infections, and malabsorption disorders.
    
#Drug Reaction:
    #High betweenness suggests that drug reactions bridge multiple medical conditions, leading to frequent confusion with infections, allergies, or autoimmune issues.
    
#These diseases connect different clusters, making them major misdiagnosis risks.
#Doctors should carefully differentiate these from diseases in both of their clusters.



In [ ]:
print("Top 5 diseases by Closeness Centrality:", sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)[:5])
#These diseases are highly reachable in the network, meaning they are common misdiagnosis or differential diagnosis candidates.
#Doctors should always consider these diseases in diagnostic decisions since they connect to many symptomatically similar conditions.
#These diseases may be important in machine learning models for automated diagnosis, as they likely appear as early-stage possibilities in differential diagnoses.



In [ ]:
print("Top 5 diseases by Eigenvector Centrality:", sorted(eigenvector_centrality.items(), key=lambda x: x[1], reverse=True)[:5])


#Since it is so interconnected, doctors should always consider checking for underlying causes of hypokalemia, as it may be a secondary symptom of another major condition.

#understanding postoperative symptom patterns could help reduce unnecessary tests and avoid confusion with unrelated conditions.

#These diseases act as key “hubs” in the network, influencing diagnosis patterns across multiple categories.
#Diseases with high eigenvector centrality may serve as key “signals” in differential diagnosis algorithms.
#Misdiagnosis risks are high when these diseases are confused with others that have similar symptoms but different underlying causes.